#### Welcome to the MPDL project

First I am going to fetch data from Springer

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("SPRINGER_API_KEY")




In [ ]:
import springernature_api_client.openaccess as openaccess



# Initialize API client
openaccess_client = openaccess.OpenAccessAPI(api_key=API_KEY)




I use climate change data in our example

In [ ]:
response = openaccess_client.search(q="climate change", p=20, s=1, fetch_all=False, is_premium=False)



In [ ]:
import pandas as pd

# Convert response to DataFrame
if "records" in response:
    df = pd.DataFrame(response["records"])
    print(df.head())

    # Save to a JSON file
    df.to_json("springer_data.json", orient="records", indent=4)

    # Save to a CSV file
    df.to_csv("springer_data.csv", index=False)

    print("Data saved successfully!")




Next I perform data cleaning

In [ ]:
import json
import pandas as pd

# Load the previously fetched Springer data
with open("springer_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Ensure data format
if isinstance(data, dict) and "records" in data:
    records = data["records"]  # Extract records list
elif isinstance(data, list):
    records = data  # Data is already a list
else:
    raise ValueError("Unexpected data format: Expected a list or dictionary with 'records' key.")

# Convert to DataFrame
df = pd.DataFrame(records)

# Handle missing keys safely
df["url"] = df["url"].apply(lambda x: x[0]["value"] if isinstance(x, list) and x else None)
df["creators"] = df["creators"].apply(lambda x: ", ".join([c.get("creator", "") for c in x]) if isinstance(x, list) else None)


def extract_subjects(x):
    if isinstance(x, list):  # Ensure x is a list
        return ", ".join([s.get("term", "") for s in x if isinstance(s, dict)])  # Extract terms safely
    elif isinstance(x, str):  # If it's a string, return it directly
        return x
    else:
        return None  # Return None for unexpected formats

df["subjects"] = df["subjects"].apply(extract_subjects)

# Select relevant columns (handle missing columns safely)
columns_to_keep = ["contentType", "identifier", "language", "url", "title", "creators", "publicationName", "subjects"]
df_cleaned = df[[col for col in columns_to_keep if col in df.columns]]

# Save cleaned data
df_cleaned.to_json("springer_data_clean.json", orient="records", indent=4)
df_cleaned.to_csv("springer_data_clean.csv", index=False)

print("Climate Change data successfully cleaned and saved!")
print(df_cleaned.head())




I have already installed PostgreSQL17 and pgAdmin on my windows machine

Start the database server (if not already). Default is port 5432

In [ ]:
import psycopg2
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()
DB_PASSWORD = os.getenv("DB_PASSWORD")

#  PostgreSQL connection details
DB_NAME = "research_metadata"
DB_USER = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"  

# Load cleaned CSV data
df = pd.read_csv("springer_data_clean.csv")

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    )
    cur = conn.cursor()
    print(" Connected to PostgreSQL!")

    #  Insert data into the database
    for _, row in df.iterrows():
        cur.execute(
            """
            INSERT INTO research_articles (content_type, identifier, language, url, title, creators, publication_name, subjects)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (identifier) DO NOTHING;
            """,
            (
                row["contentType"],
                row["identifier"],
                row["language"],
                row["url"],
                row["title"],
                row["creators"],
                row["publicationName"],
                row["subjects"],
            ),
        )

    # Commit the changes
    conn.commit()
    print("Data successfully inserted into PostgreSQL!")

except Exception as e:
    print("Database error:", e)

finally:
    cur.close()
    conn.close()
    print(" PostgreSQL connection closed.")


#### Data exploration with SQL. Run the following queries in pgAdmin

Count the total number of articles:

```

SELECT COUNT(*) FROM research_articles;
```

List the most frequent journals:

```
SELECT publication_name, COUNT(*) AS num_articles
FROM research_articles
GROUP BY publication_name
ORDER BY num_articles DESC
LIMIT 10;
```

Identify the most published authors:

```
SELECT creators, COUNT(*) AS num_articles
FROM research_articles
GROUP BY creators
ORDER BY num_articles DESC
LIMIT 10;
```


#### Data exploration with Pandas

In [ ]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load environment variables
load_dotenv()
DB_PASSWORD = os.getenv("DB_PASSWORD")

# PostgreSQL connection details
DB_NAME = "research_metadata"
DB_USER = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"

# Use SQLAlchemy to manage the database connection
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Function to Run SQL Query and Return Pandas DataFrame
def run_query(query):
    with engine.connect() as conn:
        df = pd.read_sql_query(query, conn)
    return df

# Run Queries and Analyze Data

# Total Number of Articles
total_articles = run_query("SELECT COUNT(*) FROM research_articles;")
print("Total Articles:", total_articles.iloc[0, 0])

# Top 10 Journals by Publication Count
top_journals = run_query("""
    SELECT publication_name, COUNT(*) AS num_articles
    FROM research_articles
    GROUP BY publication_name
    ORDER BY num_articles DESC
    LIMIT 10;
""")



# Top 10 Authors by Publication Count
top_authors = run_query("""
    SELECT creators, COUNT(*) AS num_articles
    FROM research_articles
    GROUP BY creators
    ORDER BY num_articles DESC
    LIMIT 10;
""")



plt.figure(figsize=(12, 6))  # Increase figure size for better readability
plt.bar(top_journals["publication_name"], top_journals["num_articles"], color="skyblue")

plt.xlabel("Journal")
plt.ylabel("Number of Articles")
plt.title("Top 10 Journals in Climate Change Research")

# Ensure all journal names are displayed properly
plt.xticks(np.arange(len(top_journals)), top_journals["publication_name"], rotation=45, ha="right")

plt.show()


